In [2]:
%matplotlib inline
import numpy as np
from attention import *
#import pop
#from models import CueAlone, Naive
from common import *
import scipy.stats as sps
from IPython.display import Image
import matplotlib.pyplot as plt

In [ ]:
-

In [ ]:
m.plot_cache_graph(filename = 'plotit.png')
Image("plotit.png")
# Cahed absent – pink #ffeeee
#Cached prsent – red #ff9999
#Ordinary absent – light blue #eeeeff
#Ordinary present - Purple #9999ff 
# Method – light green #aaccaa

#C__sn 

In [ ]:
# %load A2_fit_CG.py
# modules
from helpers import *
import attention
reload(attention)
from attention import *
import pop


# core data
#if not locals().has_key('event_data'):
#    event_data = cPickle.load(open('data/event_data.pickle'))

#event_data.load_attribute('C')

"""
=======
classes
=======
"""

class PopData( pop.PopData ):                                                                            
    
    def __init__( self, X, Y ):
        super( PopData, self ).__init__( X, Y, add_constant=False )

        
class PopSolver( pop.Factorial_k, pop.Lowpass_s, pop.ML_w, pop.PopSimple ):
    
    _k_solver_class = MopSolver

    @property
    def _max_len_h_star( self ):
        return min([ 1000, self.data.T ])
    
    @cached
    def training_idxs( data ):
        return np.isfinite( data.Y ).all( axis=1 )
    
    @cached
    def has_testing_regions():
        return False
    
    @cached
    def T_training( training_idxs ):
        return training_idxs.sum()
    
    @cached
    def T_testing():
        return 0
    
    def _slice_by_testing( self, sig ):
        return A([])
    
    def _slice_by_training( self, sig ):
        return sig[ self.training_idxs, ... ]
    
    def _zero_during_testing( self, sig ):
        sig = sig.copy()
        sig[ ~self.training_idxs, ... ] = 0
        return sig


"""
==========
how to fit
==========
"""

def fit_CG( d, check=False ):
    """ Fit S_global__tn on top of C__sn """
    # get cell
    d = d#event_data[ d_idx ]
    # is it done
    attr = 'CG'
    if check:
        return d.can_load_attribute( attr )
    if d.can_load_attribute( attr ):
        print '- already done'
        return
    # group together responses to a single trial
    d2 = d.group_by_trial
    # trial times
    t_ms = d2.event_t_msec
    min_dt = float( np.diff( t_ms ).min() )
    t_idx = np.floor( (t_ms - t_ms.min()) / min_dt ).astype(int)
    # how long is the signal
    T0 = T = t_idx.max() + 1
    T2 = int( pow(2, np.ceil(np.log2(T/2.))) ) * 2
    T3 = int( pow(2, np.ceil(np.log2(T/3.))) ) * 3
    T5 = int( pow(2, np.ceil(np.log2(T/5.))) ) * 5
    T = min(T2, T3, T5)
    # assemble spike counts
    Y__tn = d2.mean_counts_per_event.T
    # padded spike counts
    Y_padded__tn = np.zeros((T, d.N_units)) * np.nan
    Y_padded__tn[ t_idx, : ] = Y__tn
    Y__tn = Y_padded__tn
    # stimulus matrix
    X__ti = np.zeros((T, 2))
    X__ti[ t_idx[ d2.is_trial_cue_L ], 0 ] = 1
    X__ti[ t_idx[ d2.is_trial_cue_R ], 1 ] = 1
    # regression: initial conditions
    K__i = d.C.C__sn.T.flatten()
    K__i[ ~np.isfinite(K__i) ] = -3
    # initial weights
    w__n = np.ones( d.N_units )
    # remove any dodgy cells
    tok__n = (d.Y__tn.mean(axis=0) > 0.01)
    if ~tok__n.all():
        Y__tn = Y__tn[ :, tok__n ]
        w__n = w__n[ tok__n ]
        K__i = d.C.C__sn[ :, tok__n ].T.flatten()
        K__i[ ~np.isfinite(K__i) ] = -3
    # construct model data
    model_data = PopData( X__ti, Y__tn )
    # build model
    initial_conditions = { 'k': K__i, 'theta_s':[-7, 10], 's':np.zeros(T), 'w':w__n }
    model = PopSolver( model_data, initial_conditions=initial_conditions, rank=1 )
    model._grid_search_theta_s_parameters['bounds'] = [[-9, -4], [-40, 20]]
    model._bounds_s = [(-9, -4), (-60, 40)]
    # re-estimate cue conditions
    model.calc_posterior_k()
    for ks in model.k_solvers:
        if np.abs(np.diff(ks.k_vec))[0] > 1:
            ks.k_vec = np.max(ks.k_vec) * A([1, 1])
            ks.calc_posterior_k()
    model.calc_posterior_k()
    # solve for slow drift
    model.solve( verbose=2 )
    clear_output()
    # extract slow global signal
    SG__tn = model.posterior_sw.H__sw[ model.training_idxs, : ]
    # make the right shape
    trials, idxs, inv_idxs = np.unique( d.trial_idxs, return_inverse=True, return_index=True )
    SG__tn = SG__tn[ inv_idxs, : ]
    # check that it is not degenerate
    w__n = np.abs( svd( SG__tn - SG__tn.mean(axis=0)[na, :], full_matrices=False )[2][0, :].T )
    N_sig = (w__n > (w__n.max() / 10)).sum()
    if N_sig < 5:
        SG__tn *= 0
        print '**** DEGENERATE ****'
        tracer()
    if ~tok__n.all():
        S = SG__tn
        SG__tn = np.zeros( (S.shape[0], len(tok__n)) )
        SG__tn[ :, tok__n ] = S
    # done
    if d.can_load_attribute( attr ):
        print '- already done'
        return
    # save
    d.CG = Bunch(d.C.copy())
    d.CG['SG__tn'] = SG__tn
    d.CG = BaseModel( d.CG )
    d.save_attribute(attr)




In [ ]:
fit_CG( m, check=False )

In [ ]:
m.group_by_trial

In [ ]:
x = np.random.random_integers(1,2,[10,10])
y = np.mean(x,axis=1)
K = MopData(x,y)

In [ ]:
dd = {'jj':123,'kk':123}

In [131]:
class matimp(AutoCR):
    def __init__( self, copydict=None ):
        self.l = 1000
        self.N = 2
    @property 
    # instruction trials 10 first trails
    def instruct(self):
        return np.zeros([N,l])

    @property 
    #?
    def valid(self):
        return np.ones([N,l])
    
    @property 
    #?
    def catchtrial(self):
        return np.zeros([N,l])
    
    @property 
    def attendloc(self):
        return np.random.random_integers(0,1,[N,l])
    
    @property 
    def correctloc(self):
        return np.random.random_integers(0,1,[N,l])
    
    @property 
    #orientation change
    def ochange(self):
        return np.ones([N,l])
        
    
    @property 
    def eotcode(self):
        return np.zeros([l,N])
    
    @property 
    #numof stim before the change occur
    def numstim(self):
        return np.random.random_integers(10,100,[N,l])
    
    @property
    #when does trail start (overall,I guess)
    def trialstart(self):
        return np.array([100,200,500])#np.random.random_integers(1,1000,self.numstim)
    
    @property
    #
    def stimon(self):
        return self.trialstart+10
    
class TT(AutoCR):
    #A class to mimic trial information from Cohen and Maunsell
    def __init__( self, copydict=None ):
        self.l = 1000
        
    @property
    #
    def fixon(self):
        return np.array([10])
    @property
    #
    def fixate(self):
        return np.array([100])
    @property
    #in relative ms? 
    # on is usually for 200ms and off for 200-400ms 
    def stimon(self):
        return np.array([150])
    @property
    def stimoff(self):
        return np.array([300])
    @property
    # between 1000ms to 5000ms()
    def targon(self):
        return np.array([350])
    @property
    # in ms? mean RT is 260 ms(from targ on) has to be between 100–500ms
    def saccade(self):
        return np.array([400])
    @property
    def sutimes(self):
        return np.linspace(1,500,500)
    
class times(AutoCR):
    @property
    #
    def spike_counts(self):
        return np.random.poisson(1,[N,l])
    @property
    #
    def event_types(self):
        return ['standard','standard','standard']#np.random.poisson(1,[N,l])
    
    @property
    #
    def event_bounds(self):
        return np.array([1000,2000,3000])
    @property
    def N_events(self):
        return np.array([3])
 
    

In [ ]:
ti.trialstart

In [ ]:
np.random.random_integers(1,1000,3)

In [55]:

t = times = {'suarrayid':1,'sulist': np.linspace(1,l,l),'trialtimes':[tt,tt,tt] }
for i, tt in enumerate(times['trialtimes']):
        print(i,tt)
    

(0, <__main__.TT object at 0x115253450>)
(1, <__main__.TT object at 0x115253450>)
(2, <__main__.TT object at 0x115253450>)


In [57]:
t.spike_counts

AttributeError: 'dict' object has no attribute 'spike_counts'

In [ ]:
is_trial_cue_R = ti.attendloc.astype(bool)
trial_block_idx = np.cumsum(
                is_trial_cue_R[0][1:] != 
                is_trial_cue_R[0][:-1] )

trial_block_idx = conc([ [0], trial_block_idx ])
trial_block_idx = trial_block_idx

# trial within the block
trial_idx_within_block = conc([
    np.arange(np.sum(trial_block_idx == i)) 
    for i in range(np.max(trial_block_idx) + 1) ])
    

In [ ]:
ti.trialstart[i],
self.is_trial_instruction[0][i],
self.is_trial_valid[0][i]
self.is_trial_catch[0][i]
self.is_trial_cue_R[0][i]
self.is_trial_target_R[0][i]
self.trial_end_code[0][i]
self.trial_block_idx[i]
self.trial_idx_within_block[i]
self.trial_N_stimuli_presented[i]
self.target_orientation_change[0][i] 

In [132]:
from RawData import *
reload(RawData)

<module 'RawData' from 'RawData.pyc'>

In [133]:

l = 1000
ti = matimp()
tt = TT()
t = times()
a = RawDataSim(l,N,TT,ti,t,keep_raw=False, copydict=None )


In [ ]:

a.filter_units( a.all_event_data.Y__tn.mean( axis=0 ) > 0.1 ) 

> /Users/olegvinogradov/Documents/MasterThesis/attention/RawData.py(310)all_event_data()
    308         # event details
    309         from IPython.core.debugger import Tracer; Tracer()()
--> 310         e.spike_counts = np.hstack([ t.spike_counts for t in self ])
    311         e.event_types = np.hstack([ t.event_types for t in self ])
    312         e.event_t_msec = np.hstack( self.t_msec_absolute_stimulus_on )

ipdb> np.hstack([ t.event_types for t in self ])
*** TypeError: unhashable type: 'numpy.ndarray'
ipdb> np.hstack( self.t_msec_absolute_stimulus_on )
array([110, 210, 510])
ipdb> np.hstack([ t.event_types for t in self ])
*** TypeError: unhashable type: 'numpy.ndarray'
ipdb>  e.event_types
*** IndentationError: unexpected indent (<stdin>, line 1)
ipdb> e.event_types
*** AttributeError: 'EventData' object has no attribute 'event_types'
ipdb>  t.event_types for t in self 
*** SyntaxError: invalid syntax (<stdin>, line 1)
ipdb> ls
*** NameError: name 'ls' is not defined
ipdb>

In [ ]:
 t = times = {'suarrayid':[1,2],'sulist': [np.linspace(1,l,l), np.linspace(1,l,l)],'trialtimes':[TT(),TT(),TT()]}

In [ ]:
udiag(x)

In [ ]:
C__sn = m.C__sn.T
G__tn = np.random.random_integers(1,2,[1000,2])
M = Naive(v,G__tn = G__tn,C__sn = C__sn)
    
    

In [ ]:
M.plot_cache_graph(filename = 'plotit.png')
Image("plotit.png")

In [ ]:
x = np.array([1,2,3])

In [ ]:
x1 = np.array([x,na])
x2 = np.array([x,na,na])

In [ ]:
print(x1.shape)
print(x.shape)
print(x2.shape)

In [ ]:
from mop import *
from autoreload import *
from autocr import *
import numpy as np

In [ ]:
X = np.random.normal(0,1,[4,100])
y = np.mean(X,axis=1)
a = Data(X,y)

In [ ]:
S = UnitSolver(a)

In [ ]:
a.create_postrior()

In [ ]:
plt.figure(figsize=(50,30))
plt.imshow(v.Y__tn.T)
#plt.plot(v.S__ts,'r')
#plt.plot(np.mean(v.D,1),'b')
#plt.plot(v.M,'g')
plt.plot(v.a,'k')


In [ ]:
# %load A2_fit_CG
# modules
from helpers import *
import attention
reload(attention)
from attention import *
import mop

# core data
#if not locals().has_key('event_data'):
#    event_data = cPickle.load(open('data/event_data.pickle'))
#
#event_data.load_attribute('C')

"""
=======
classes
=======


class PopData( pop.PopData ):                                                                            
    
    def __init__( self, X, Y ):
        super( PopData, self ).__init__( X, Y, add_constant=False )

        
class PopSolver( pop.Factorial_k, pop.Lowpass_s, pop.ML_w, pop.PopSimple ):
    
    _k_solver_class = MopSolver

    @property
    def _max_len_h_star( self ):
        return min([ 1000, self.data.T ])
    
    @cached
    def training_idxs( data ):
        return np.isfinite( data.Y ).all( axis=1 )
    
    @cached
    def has_testing_regions():
        return False
    
    @cached
    def T_training( training_idxs ):
        return training_idxs.sum()
    
    @cached
    def T_testing():
        return 0
    
    def _slice_by_testing( self, sig ):
        return A([])
    
    def _slice_by_training( self, sig ):
        return sig[ self.training_idxs, ... ]
    
    def _zero_during_testing( self, sig ):
        sig = sig.copy()
        sig[ ~self.training_idxs, ... ] = 0
        return sig

"""
"""
==========
how to fit
==========
"""

def fit_CG( d, check=False ):
    """ Fit S_global__tn on top of C__sn """
    # get cell
    #d = event_data[ d_idx ]
    # is it done
    attr = 'CG'
    #if check:
    #    return d.can_load_attribute( attr )
    # if d.can_load_attribute( attr ):
    #     print '- already done'
    #    return
    # group together responses to a single trial
    d2 = d#.group_by_trial
    # trial times
    t_ms = d2.event_t_msec
    min_dt = float( np.diff( t_ms ).min() )
    t_idx = np.floor( (t_ms - t_ms.min()) / min_dt ).astype(int)
    # how long is the signal
    T0 = T = t_idx.max() + 1
    T2 = int( pow(2, np.ceil(np.log2(T/2.))) ) * 2
    T3 = int( pow(2, np.ceil(np.log2(T/3.))) ) * 3
    T5 = int( pow(2, np.ceil(np.log2(T/5.))) ) * 5
    T = min(T2, T3, T5)
    # assemble spike counts
    Y__tn = d2.mean_counts_per_event.T
    # padded spike counts
    Y_padded__tn = np.zeros((T, d.N_units)) * np.nan
    Y_padded__tn[ t_idx, : ] = Y__tn
    Y__tn = Y_padded__tn
    # stimulus matrix
    X__ti = np.zeros((T, 2))
    X__ti[ t_idx[ d2.is_trial_cue_L ], 0 ] = 1
    X__ti[ t_idx[ d2.is_trial_cue_R ], 1 ] = 1
    
    
    # regression: initial conditions
    K__i = d.C.C__sn.T.flatten()
    K__i[ ~np.isfinite(K__i) ] = -3
    # initial weights
    w__n = np.ones( d.N_units )
    # remove any dodgy cells
    tok__n = (d.Y__tn.mean(axis=0) > 0.01)
    if ~tok__n.all():
        Y__tn = Y__tn[ :, tok__n ]
        w__n = w__n[ tok__n ]
        K__i = d.C.C__sn[ :, tok__n ].T.flatten()
        K__i[ ~np.isfinite(K__i) ] = -3
    # construct model data
    model_data = MopData( X__ti, Y__tn )
    # build model
    initial_conditions = { 'k': K__i, 'theta_s':[-7, 10], 's':np.zeros(T), 'w':w__n }
    model = PopSolver( model_data, initial_conditions=initial_conditions, rank=1 )
    model._grid_search_theta_s_parameters['bounds'] = [[-9, -4], [-40, 20]]
    model._bounds_s = [(-9, -4), (-60, 40)]
    # re-estimate cue conditions
    model.calc_posterior_k()
    for ks in model.k_solvers:
        if np.abs(np.diff(ks.k_vec))[0] > 1:
            ks.k_vec = np.max(ks.k_vec) * A([1, 1])
            ks.calc_posterior_k()
    model.calc_posterior_k()
    # solve for slow drift
    model.solve( verbose=2 )
    clear_output()
    # extract slow global signal
    SG__tn = model.posterior_sw.H__sw[ model.training_idxs, : ]
    # make the right shape
    trials, idxs, inv_idxs = np.unique( d.trial_idxs, return_inverse=True, return_index=True )
    SG__tn = SG__tn[ inv_idxs, : ]
    # check that it is not degenerate
    w__n = np.abs( svd( SG__tn - SG__tn.mean(axis=0)[na, :], full_matrices=False )[2][0, :].T )
    N_sig = (w__n > (w__n.max() / 10)).sum()
    if N_sig < 5:
        SG__tn *= 0
        print '**** DEGENERATE ****'
        tracer()
    if ~tok__n.all():
        S = SG__tn
        SG__tn = np.zeros( (S.shape[0], len(tok__n)) )
        SG__tn[ :, tok__n ] = S
    # done
    if d.can_load_attribute( attr ):
        print '- already done'
        return
    # save
    d.CG = Bunch(d.C.copy())
    d.CG['SG__tn'] = SG__tn
    d.CG = BaseModel( d.CG )
    d.save_attribute(attr)

In [ ]:
fit_CG(v)

In [ ]:
#Fit fro drift and cue
# require gain  and cue
#self.G__tn = exp( d.H__tn ) #from time_course
C__sn = m.C__sn
G__tn = np.random.random_integers(1,2,[10,1])
m = Naive(v,tau=2, rank=1, eps_per_obs=1e-5,
          max_admm_iterations=100, max_newton_steps=10, 
          nonlinearity='exp',nonlinearity_h='exp',
          G__tn=G__tn ,C__sn= C__sn, verbose= True)

In [ ]:
np.array([v.is_trial_cue_L.mean(),v.is_trial_cue_R.mean()])

In [ ]:
 np.array([v.is_trial_cue_L.mean(), v.is_trial_cue_R.mean()])

In [ ]:
m.LL_per_obs

In [ ]:
# %load A4_simulation.py
import attention
reload(attention)
from attention import *

#target_filename = 'data/sim_event_data.v3.pickle'
true_ranks = [1, 2]
N_samples = 2

#event_data = cPickle.load(open('data/event_data.pickle'))


### SIMULATE


# get cell
#d = event_data[13].__copy__()
N, T = 2, 100

#if os.path.exists(target_filename):
#    print 'loading...'
#    sim_event_data = cPickle.load(open(target_filename))

#else:
 #   print 'calculating...'
    # source model
    #d.load_attribute(['CGL', 'CGLM_2'])
    #m = Model( d.CGLM_2 )

    # parameters
gain = 0.137

    # new cells
dsims = []
for rank in progress.numbers( true_ranks ):
    for sample_idx in progress.dots( range(N_samples), 'rank = %d' % rank ):
        # sample W - weigths of the modulator
        W__in = np.random.normal(0, 1, size=(rank, N) )
        # sample S - stimuli
        S__ti = np.random.normal(0,1, size=(T, rank) )
        # create Omega
        Omega__tn = S__ti.dot(W__in)
        # reweight
        Omega__tn -= Omega__tn.mean()
        # multiply by gain
        Omega__tn *= gain / Omega__tn.std()
        # mean rate
        Mu__tn = 1* np.exp( Omega__tn + 0.1 )
        # sample
        Y__tn = np.random.poisson( Mu__tn )
        # replace
        #this_d = d#.__copy__()
        #this_d.Y__tn = Y__tn.T
        #this_d.prefix = 'sim_v3_rank_%d_sample_%02d' % (rank, sample_idx)
        # remove keys
        #to_remove = ['trial_end_code', 'prev_trial_end_code', 'event_idxs_within_trial', 'N_events_within_trial', 'trial_idxs_within_block', 'is_trial_catch', 'prev_prev_trial_target_R', 'target_orientation_change', 'prev_prev_trial_catch', 'prev_prev_trial_end_code', 'prev_trial_catch', 'prev_trial_target_R', 'trial_block_idxs', 'trial_idxs']
        #for k in to_remove:
        # delattr( this_d, k )
        # save
        #dsims.append( this_d )

#sim_event_data = EventDataList( dsims )

# save
#cPickle.dump( sim_event_data, open(target_filename, 'w'), 2 )


### COPY OVER PARAMETERS





In [ ]:
plt.plot(Mu__tn)

In [ ]:
sim_event_data[1].Y__tn[0].shape

In [ ]:
plt.figure(0)
plt.plot(sim_event_data[3].Y__tn.T)
plt.plot(sim_event_data[3].S__ts)


In [ ]:
# %load A1_fit_C.py
# modules
import attention
reload(attention)
from attention import *
#import pop



# fit CueAlone model
i=0
for d in progress.numbers( sim_event_data ):
    #print(d.C.C__sn)
    m = models.CueAlone( d, verbose=False )
    print(m.C__sn)
    #sim_event_data[i].C = BaseModel({'C__sn':m.C__sn})
    #i+=1
    #d.save_attribute('C')


In [ ]:
sim_event_data[0].__dict__.keys() 
sim_event_data[50].C

In [ ]:
# %load A1_fit_C.py
# modules
import attention
reload(attention)
from attention import *
import pop

# core data
if not locals().has_key('event_data'):
    event_data = cPickle.load(open('data/event_data.pickle'))

# fit CueAlone model
for d in progress.numbers( event_data ):
    m = models.CueAlone( d, verbose=False )
    d.C = BaseModel({'C__sn':m.C__sn})
    d.save_attribute('C')


In [ ]:
# %load A1_fit_C.py
# modules
import attention
reload(attention)
from attention import *
import mop
import numpy as np

In [ ]:
for d in progress.numbers(v):
    print(d)
    m = models.CueAlone( d, verbose=True )

In [ ]:
m

In [ ]:
(v[1].Y__tn.T.dot(v[1].S__ts)/v[1].S__ts(axis=0)[None,:]).T

In [ ]:
A = np.array
k = A([[1,1,1],[1,1,1]]).T.astype(float)

In [ ]:
print(k.sum(axis=0))

In [ ]:
v[1].N_events

In [ ]:
a =EventData(1,3)

In [ ]:
v[1].Y__tn

In [ ]:
for each in c: print(each)

In [ ]:
n=0
S__ts = v[1].S__ts
Nu__sn = ( v[1].Y__tn.T.dot(v[1].S__ts) / v[1].S__ts.sum(axis=0)[None, :] ).T
C__sn = log( Nu__sn )
z_vec = C__sn[:, n]
S=2
a__s = z_vec[-S:]
nu__t = exp( S__ts.dot(a__s) )



In [ ]:
C__sn


In [ ]:
v[1].Y__tn.T.dot(v[1].S__ts) 

In [ ]:
d

In [ ]:
import numpy as np

In [ ]:
A = np.array